In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from model.classes import network
from model import helpers as hlp
from halfmoon import halfmoon_helpers as hmhlp

In [ ]:
hm = hmhlp.halfMoon(200, 10, -5, 6)
# hm = hmhlp.halfMoon(200, 10, 1, 6)

In [ ]:
hmhlp.plotHalfMoon(hm[0], hm[1])

In [ ]:
df_pts = pd.DataFrame(hm[0], columns=['x', 'y'])
df_moons = pd.DataFrame(hm[1], columns=['in_red'])

In [ ]:
df = pd.concat([df_pts, df_moons], axis=1, join='inner')

In [ ]:
x_cols = ['x', 'y']
y_cols = ['in_red']

df_train_x, df_train_y = hlp.split_input_output_data(df, x_cols=x_cols, y_cols=y_cols)

In [ ]:
df_train_x_norm = hlp.normalize(df_train_x, -15, 25, -1, 1)
df_val_x_norm = hlp.normalize(df_train_x, -15, 25, -1, 1)

In [ ]:
mdl_hm = network.Feed_Forward()
mdl_hm.add_layer(2)
mdl_hm.add_layer(5, activation_function='sigmoid')
mdl_hm.add_layer(1, activation_function='sigmoid')

In [ ]:
mdl_hm.plot_network()

In [ ]:
history = mdl_hm.train(
    df_train_x_norm,
    df_train_y,
    epochs=1000,
    shuffle=True,
    debug=True,
    max_error=0.0001,
    momentum_factor=0.6
)
    
hlp.plot_learning_curve(history)

In [ ]:
list_pts = df_pts.values.tolist()
list_moons = df_moons.values.tolist()

for l in range(len(list_pts)):
    p = 0 if mdl_hm.predict(list_pts[l])[0] < 0.5 else 1
    list_pts[l].append(p)
    list_pts[l].append(int(list_moons[l][0]))

In [ ]:
arr = []

min_val_1 = -15
max_val_1 = 25
min_val_2 = -10
max_val_2 = 15

upper_bound = 1
lower_bound = -1

for i in range(min_val_1, max_val_1):
    for j in range(min_val_2, max_val_2):
        i_norm = (i - min_val_1) / (max_val_1 - min_val_1) * (upper_bound - lower_bound) + lower_bound
        j_norm = (j - min_val_2) / (max_val_2 - min_val_2) * (upper_bound - lower_bound) + lower_bound
        p = mdl_hm.predict([i_norm, j_norm])[0]
        arr.append([i, j, p])

In [ ]:
np_array = np.array(arr)
transpose = np_array.T
transpose_list = transpose.tolist()

transpose_2 = hm[0].T
transpose_list_2 = transpose_2.tolist()

In [ ]:
cm = plt.cm.get_cmap('bwr')

plt.figure(1)
plt.scatter(transpose_list[0], transpose_list[1], c=transpose_list[2], marker='o', cmap=cm, alpha=0.5)
plt.scatter(transpose_list_2[0], transpose_list_2[1], c='black', marker='x')

In [ ]:
mdl_hm.layers[1].get_weights()

In [ ]:
mdl_hm.layers[2].get_weights()